**EXERCISE 1**

The goal of this exercise is to get into some first contact with Python, Pandas data frames, and the many opportunities how to quickly generate static charts from data frames.

For the first exercise, load the provided CSV file, do some data cleaning, and check if your cleaning operation was successful using visualization. For inspiration, browse through some Kaggle notebooks. The original data was downloaded from Kaggle. 

The main purpose of the notebook is to preprocess the CSV file for further visualization. The following steps have to be performed: 

Read the three CSV using Pandas. See the pandas.read_csv documentation to check how to parse the CSV correctly! Merge the two datasets player_data_per_36_min.csv and players.csv into one table. Add a team_name column to your new data frame based on the team_id column retrieving the name from the teams.csv dataset. You should now have a table that includes player info from players.csv, and players performance statistics from player_data_per_36_min.csv, and the team’s name from teams.csv. (3 points)
Take care of missing values. Sci-kit learn provides different data imputation methods. Remove unusable rows or columns, if necessary.  If the player has no current team, replace the empty value with "Retired".
(Hint: You may need to impute before you completely filter your dataframe. You need to make an educated judgment). (3 points)
Create two new tables: one that groups rows based on player_id, another that groups rows based on team_id. (2 points)
Visualize the data (twice). Every submitted notebook should contain at least two visualizations using at least two different Python visualization libraries. One visualization for each of the new tables (aggregated players or aggregated teams). A list of the most wide-spread Python visualization libraries can be found in this article. You must concisely describe and explain each visualization and your decisions in a Markdown field. You will not receive the points for this task if you did not add a description/explanation. (max. 5 points per visualization)
Save the resulting tables (the cleaned player_data_per_36_min and the two new aggregated tables) as CSV. To be sure that the data is correctly saved, you can load it again. You will have to work with these tables for the second exercise. (2 points)
Possible visualizations include, but are not limited to: 

Scatterplots and scatterplot matrices
Parallel coordinates
Radar charts
Bar charts
Box plots and histograms 
Choropleth maps
...

# ToDo
- map plot number of players and states

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import plotly.express as px

Read data

In [3]:
players = pd.read_csv("data/players.csv")
teams = pd.read_csv("data/teams.csv")
player_data_per_36_min = pd.read_csv("data/player_data_per_36_min.csv")

Check for duplicates

In [5]:
print(
    f"Duplikate in Player (36 min): {len(player_data_per_36_min[player_data_per_36_min.duplicated()==True])}"
)
print(f"Duplikate in Player: {len(players[players.duplicated()==True])}")
print(f"Duplikate in Player: {len(teams[teams.duplicated()==True])}")

Duplikate in Player: 0
Duplikate in Player: 0
Duplikate in Player: 0


In [7]:
players[players.duplicated()==True]

,id,name,full_name,position,height,weight,birth_date,birth_place,retired,player_url,current_team_id,teams,text


In [8]:
players.shape

(4820, 13)

In [9]:
teams[teams.duplicated()==True]

,team_id,team_name,short_name,years,total_games,total_wins,total_losses,win_loss_percentage,champions,other_names,link,active


In [10]:
teams.shape

(52, 12)

In [11]:
df1 = players.merge(
    player_data_per_36_min, how="right", left_on="id", right_on="player_id"
)

In [12]:
df2 = df1.merge(
    teams[["team_name", "team_id"]],
    how="left",
    left_on="current_team_id",
    right_on="team_id",
)

In [13]:
df2 = df2.replace({np.nan: None})

In [14]:
df2.loc[df2["retired"], "team_name"] = "Retired"

In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26674 entries, 0 to 26673
Data columns (total 42 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_x             26674 non-null  int64 
 1   name             26674 non-null  object
 2   full_name        26674 non-null  object
 3   position         26674 non-null  object
 4   height           26674 non-null  object
 5   weight           26671 non-null  object
 6   birth_date       26673 non-null  object
 7   birth_place      26278 non-null  object
 8   retired          26674 non-null  bool  
 9   player_url       26674 non-null  object
 10  current_team_id  2830 non-null   object
 11  teams            26674 non-null  object
 12  text             26674 non-null  object
 13  id_y             26674 non-null  int64 
 14  player_id        26674 non-null  int64 
 15  season           26674 non-null  object
 16  total_games      26645 non-null  object
 17  games_started    20208 non-null

In [16]:
birth_places = df2["birth_place"].unique()

In [17]:
df2["birth_states"] = df2["birth_place"].str.split(',', expand=True)[1]
df2['birth_states'] = df2['birth_states'].apply(lambda x: str(x).replace(u'\xa0', u''))


In [18]:
df2["birth_states"].unique()

array(['Indiana', 'New York', 'Kentucky', 'Wisconsin', 'Idaho',
       'Pennsylvania', 'New Jersey', 'Michigan', 'Oklahoma', 'Kansas',
       'Louisiana', 'Texas', 'Ohio', 'Illinois', 'California',
       'Minnesota', 'Montana', 'Tennessee', 'Wyoming', 'Georgia',
       'Colorado', 'None', 'Utah', 'Iowa', 'Washington', 'Maryland',
       'Oregon', 'West Virginia', 'North Dakota', 'Poland',
       'Massachusetts', 'Missouri', 'North Carolina', '', 'Hawaii',
       'Nebraska', 'Arizona', 'Canada', 'Alabama', 'Arkansas',
       'South Dakota', 'Virginia', 'Mississippi', 'South Carolina',
       'Connecticut', 'United Kingdom', 'Florida', 'District of Columbia',
       'China', 'New Mexico', 'Delaware', 'Germany', 'Rhode Island',
       'Morocco', 'Netherlands', 'Nevada', 'Romania', 'Denmark',
       'Puerto Rico', 'Bahamas', 'Panama', 'Iceland', 'France', 'Spain',
       'Nigeria', 'Jamaica', 'Maine', 'South Sudan', 'Bulgaria', 'Haiti',
       'Brazil', 'Dominican Republic', 'Lebanon', 'N

In [19]:
all_states = pd.Series(df2["birth_states"].unique())

In [20]:
US_states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
    'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
    'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
    'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
    'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
    'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
    'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
    'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
    'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
    'West Virginia', 'Wisconsin', 'Wyoming'
]


In [21]:
len(US_states)

50

Es gab bis 2019 nie einen NBA Spieler aus Vermont 

In [22]:
np.setdiff1d(US_states, all_states[pd.Series(all_states).isin(US_states)].unique())

array(['Vermont'], dtype='<U14')

In [23]:
df2_US = df2.loc[df2["birth_states"].isin(US_states)]

In [24]:
df2_US_map = df2_US[["birth_states", "player_id"]].groupby(["birth_states"]).count().reset_index()

In [25]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

fig = px.choropleth_mapbox(df2_US_map, geojson=counties, locations='birth_states', color='player_id',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           #zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'num:_players':'number of players'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
df2["pts"].isna().value

0         True
1         True
2         True
3         True
4         True
         ...  
26669    False
26670    False
26671    False
26672    False
26673    False
Name: pts, Length: 26674, dtype: bool